<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/extract_images_tumorAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Extract_images_tumorAI**

In [1]:
# eyelid_conj_tumor_listを先に日本語に直しておくと後が楽だった...
patient_list_path = r"F:\SlitPhoto\20100101~20230130_スリット(写真室)\eyelid_conj_tumor_list.csv"
IDMap_path = r"F:\SlitPhoto\20100101~20230130_スリット(写真室)\combined_IDMap.csv"

In [ ]:
import pandas as pd

# Load the first CSV file
df1 = pd.read_csv(patient_list_path, encoding='cp932')

# Load the second CSV file
df2 = pd.read_csv(IDMap_path, encoding='cp932', header = None)

# Displaying the first few rows of each DataFrame to understand their structure
df1.head(), df2.head()


In [ ]:
df1

In [ ]:
df1 = df1.drop_duplicates(subset='ID')
df1

In [ ]:
df2

In [7]:
# df2[df2[0]==11845831].loc[0,2]
df2[df2[0]==5815065].loc[:,2]

Series([], Name: 2, dtype: object)

In [ ]:
import os
import pandas as pd
from datetime import datetime
import shutil

def format_date(date_str):
    # Convert to datetime object
    date_obj = datetime.strptime(date_str, "%Y/%m/%d")
    # Convert to YYYYMMDD format string
    return date_obj.strftime("%Y%m%d")

def copy_eligible_images(src, dst, img_file):
    """
    Copies eligible images from source to destination folder.
    Creates destination folder if it does not exist.
    """
    if not os.path.exists(dst):
        os.makedirs(dst)
    shutil.copy(os.path.join(src, img_file), dst)

# Initialize a counter
hash_value_count = 0

# Image folder paths
img_folders = [
    r"F:\SlitPhoto\20100101~20230130_スリット(写真室)\ID_00######\ZZZ\Files",
    r"F:\SlitPhoto\20100101~20230130_スリット(写真室)\ID_09######\ZZZ\Files",
    r"F:\SlitPhoto\20100101~20230130_スリット(写真室)\ID_10######\ZZZ\Files",
    r"F:\SlitPhoto\20100101~20230130_スリット(写真室)\ID_11######\ZZZ\Files",
    r"F:\SlitPhoto\20100101~20230130_スリット(写真室)\ID_12######\ZZZ\Files"
]

dst_path = r"F:\SlitPhoto\Extracted_tumor_images"
patient_num = 0

for user_id in df1["ID"]:
    try:
        hash_value = df2[df2[0] == user_id].iloc[0, 2]
        print(hash_value)
        hash_value_count += 1

        exam_date = df1[df1["ID"] == user_id]["ExamDate"].iloc[0]
        formatted_date = format_date(exam_date)
        print(f"Formatted ExamDate for {user_id}: {formatted_date}")

        folder_found = False
        for folder in img_folders:
            for file in os.listdir(folder):
                if file.startswith(hash_value):
                    folder_found = True
                    for img_file in os.listdir(os.path.join(folder, file)):
                        try:
                            if int(img_file[:8]) <= int(formatted_date):
                                print(f"Eligible image file: {os.path.join(folder, file, img_file)}")
                                # New code to copy files
                                area = df1[df1["ID"] == user_id]["Area"].iloc[0]
                                diag_classification = df1[df1["ID"] == user_id]["Diag_classification"].iloc[0]
                                dst_folder = f"{dst_path}/OSK_{area}{diag_classification}_症例{patient_num}"
                                copy_eligible_images(os.path.join(folder, file), dst_folder, img_file)
                        except ValueError:
                            continue
                    patient_num += 1
                    break
            if folder_found:
                break
    except Exception as e:
        print(f"No image file in user ID {user_id}: {e}")
        pass
print("Total hash values printed:", hash_value_count)

In [ ]:
"""
フォルダ別に画像が出力されるので、不適切な画像の削除、不適切なフォルダの削除、リネームを行う

"""

In [68]:
orig_path = r"F:\SlitPhoto\Extracted_tumor_images"
dst_path = r"F:\SlitPhoto\Extracted_tumor_images_sorted"

In [75]:
os.listdir(orig_path)[0]

'OSK_EyelidSebaceous gland carcinoma_症例0'

In [90]:
def check_duplicate_filenames_with_folders(path):
    # 初期化: 全ファイル名を格納する辞書と重複ファイルを格納する辞書
    all_filenames = {}
    duplicates = {}

    # 指定したパスからすべてのサブディレクトリとファイルを探索
    for root, dirs, files in os.walk(path):
        # 各ファイルに対して重複チェックを行う
        for filename in files:
            if filename in all_filenames:
                # 重複が見つかった場合、重複リストに追加（どのフォルダと重複しているかも含む）
                if filename not in duplicates:
                    duplicates[filename] = [all_filenames[filename]]
                duplicates[filename].append(root)
            else:
                # 重複がない場合は、ファイル名とそのフォルダを記録
                all_filenames[filename] = root

    # 重複しているファイルの情報を含む辞書を返す
    return duplicates

# 再び仮想のパスでチェックを実行
# orig_pathは、この関数を呼び出す前に定義されている必要がある
duplicates_with_folders = check_duplicate_filenames_with_folders(orig_path)
duplicates_with_folders


{}

In [93]:
# Number of cases
print(f"Number of cases: {len(os.listdir(orig_path))}")

Nubber of cases: 332


In [ ]:
"""
送付画像の形式：
OSK_{disease}_症例{patient_num}_画像{image_num}
"""


In [94]:
#別フォルダに画像をコピー
import shutil
import os
from tqdm import tqdm

# Paths for the original and destination directories
orig_path = r"F:\SlitPhoto\Extracted_tumor_images"
dst_path = r"F:\SlitPhoto\Extracted_tumor_images_sorted"

# Getting the list of folders in orig_path
folders = [f for f in os.listdir(orig_path) if os.path.isdir(os.path.join(orig_path, f))]

# Copying each folder from orig_path to dst_path with a progress bar
for folder_name in tqdm(folders, desc="Copying folders"):
    src_folder = os.path.join(orig_path, folder_name)
    dst_folder = os.path.join(dst_path, folder_name)
    shutil.copytree(src_folder, dst_folder)

In [ ]:
os.listdir(dst_path)

In [ ]:
#英語の病名を日本語に変換import os

# Path to the destination directory
dst_path = r"F:\SlitPhoto\Extracted_tumor_images_sorted"

# List of tuples with (string to find, string to replace)
replacement_pairs = [
    ('Pterygium', '翼状片'),
    ('CorneaOther Benign lesion', 'その他良性病変'),
    ('EyelidCyst', '結膜嚢胞'),
    ('ConjunctivaLymphoma', '悪性リンパ腫'),
    ('EyelidCyst', '眼瞼嚢胞'),
    ('EyelidNevus', '眼瞼母斑'),
    ('EyelidBasal cell carcinoma', '基底細胞癌'),
    ('EyelidChalazion', '霰粒腫'),
    ('ConjunctivaChalazion', '霰粒腫'),
    ('ConjunctivaVascular Malformation', '血管奇形'),
    ('EyelidVascular Malformation', '血管奇形'),

    ('ConjunctivaOther Benign lesion', 'その他良性病変'),
    ('EyelidOther Benign lesion', 'その他良性病変'),
    ('Chalazion', '霰粒腫'),
    ('ConjunctivaCyst', '結膜嚢胞'),
    ('ConjunctivaSebaceousGlandcarcinoma', '脂腺癌'),
    ('ConjunctivaIntraepithelial neoplasia', '結膜上皮内癌'),
    ('EyelidBasal cell carcinoma', '基底細胞癌'),
    ('ConjunctivaNevus', '結膜母斑'),
    ('ConjunctivaPapilloma', '乳頭腫'),
    ('CorneaDermoid', 'デルモイド'),
    ('ConjunctivalDermoid', 'デルモイド'),

    ('ConjunctivaOther benign lesion', 'その他良性病変'),
    ('ConjunctivaOther carcinoma', '結膜癌その他'),
    ('Eyelid_Sebaceous Gland Carcinoma', '脂腺癌'),
    ('EyelidSebaceous Gland Carcinoma', '脂腺癌'),
    ('ConjunctivaMelanoma', '悪性黒色腫'),
    ('ConjunctivaEpidermal cyst', 'アテローム'),
    ('EyelidEpidermal cyst', 'アテローム'),
    ('EyelidGranulation', '肉芽腫'),
    ('ConjunctivaGranulation', '肉芽腫'),
    ('ConjunctivaFat hernia', '眼窩脂肪ヘルニア'),
    ('ConjunctivaFatHernia', '眼窩脂肪ヘルニア'),
    ('ConjunctivaDermoid', 'デルモイド'),

    ('EyelidSebaceous gland carcinoma', '脂腺癌'),
    ('CorneaOther carcinoma', '結膜癌その他'),
    ('EyelidCharazion', '霰粒腫'),
    ('EyelidLymphoma', '悪性リンパ腫'),
    ('ConjunctivaGlanulation', '肉芽腫'),
    ('ConjunctivaHordeolum', '麦粒腫'),
    ('EyelidOther benign lesion', 'その他良性病変'),
    ('EyelidPapilloma', '乳頭腫'),
    ('ConjunctivaSquamous cell carcinoma', '扁平上皮癌'),
    ('EyelidSeborrheic keratosis', '脂漏性角化症'),
    ('EyelidSebaceous gland carcinoma', '脂腺癌'),

    ('EyelidOther carcinoma', '眼瞼癌その他'),
    ('ConjunctivaPigmentation', '結膜色素沈着'),
    ('Cornea翼状片', '翼状片'),
    ('ConjunctivaOtherBenignLesion', 'その他良性病変'),
    ('EyelidSquamous cell carcinoma', '扁平上皮癌'),
    ('EyelidVascular malformation', '血管奇形'),
    ('ConjunctivaLymphom', '悪性リンパ腫'),
    ('ConjunctivaPrimaryAcquiredMelanosis', '結膜色素沈着(PAM)'),
    ('ConjunctivalPapilloma', '乳頭腫'),
    ('ConjunctivaIntraepithelialNeoplasia', '結膜上皮内癌'),
    ('EyelidSebaceousGlandCarcinoma', '脂腺癌'),
    ('EyelidMelanoma', '脂腺癌'),
    # Add more pairs as needed, e.g., ('old_string', 'new_string'),
]


# Iterate over the folders in dst_path
for folder_name in os.listdir(dst_path):
    new_folder_name = folder_name

    # Check and replace each pair in the folder name
    for old_string, new_string in replacement_pairs:
        new_folder_name = new_folder_name.replace(old_string, new_string)

    # If the name has changed, rename the folder
    if new_folder_name != folder_name:
        old_folder_path = os.path.join(dst_path, folder_name)
        new_folder_path = os.path.join(dst_path, new_folder_name)

        os.rename(old_folder_path, new_folder_path)
        print(f"Renamed '{folder_name}' to '{new_folder_name}'")



In [ ]:
os.listdir(dst_path)

In [ ]:
"""
日本語の病名に修正したので、送付用にリネーム
OSK_アテローム_症例127 --> OSK_アテローム_症例127_画像n

※症例番号、画像番号はゼロ埋めで3桁
※症例番号は000からふり直し（欠番を防ぐため）
"""

In [123]:
#確認用
import os
import re
import shutil

def rename_and_copy_files(orig_path, dst_path):
    for folder_name in os.listdir(orig_path):
        folder_path = os.path.join(orig_path, folder_name)
        if os.path.isdir(folder_path):
            # Extract and zero-pad the case number
            case_number_match = re.search(r'症例(\d+)', folder_name)
            if case_number_match:
                case_number = case_number_match.group(1).zfill(3)
                new_folder_name = re.sub(r'症例\d+', f'症例{case_number}', folder_name)
            else:
                new_folder_name = folder_name

            for i, file_name in enumerate(os.listdir(folder_path), 1):
                new_name = f"{new_folder_name}_画像{str(i).zfill(3)}{os.path.splitext(file_name)[1]}"
                src_file_path = os.path.join(folder_path, file_name)
                dst_file_path = os.path.join(dst_path, new_name)
                shutil.copy2(src_file_path, dst_file_path)

orig_path = r"F:\SlitPhoto\Extracted_tumor_images_sorted"
dst_path = r"F:\SlitPhoto\Extracted_tumor_images_送付用"

if os.path.exists(dst_path):
    shutil.rmtree(dst_path)
os.makedirs(dst_path)

rename_and_copy_files(orig_path, dst_path)


In [124]:
#症例番号ふり直しバージョン
#正常も番号振ってコピー

import os
import shutil
import re

def rename_and_copy_files(orig_path, normal_path, dst_path):
    # Create a mapping for each unique disease to a new case number
    disease_to_case_number = {}

    # Process tumor images
    for folder_name in os.listdir(orig_path):
        folder_path = os.path.join(orig_path, folder_name)
        if os.path.isdir(folder_path):
            # Extract disease name from the folder
            disease_name_match = re.search(r'(.+)_症例\d+', folder_name)
            if disease_name_match:
                disease_name = disease_name_match.group(1)
                # Initialize or increment the case number for this disease
                disease_to_case_number[disease_name] = disease_to_case_number.get(disease_name, -1) + 1
                new_case_number = f"症例{str(disease_to_case_number[disease_name]).zfill(3)}"

                for i, file_name in enumerate(os.listdir(folder_path), 1):
                    new_name = f"{disease_name}_{new_case_number}_画像{str(i).zfill(3)}{os.path.splitext(file_name)[1]}"
                    src_file_path = os.path.join(folder_path, file_name)
                    dst_file_path = os.path.join(dst_path, new_name)
                    shutil.copy2(src_file_path, dst_file_path)

    # Process normal images
    for i, file_name in enumerate(os.listdir(normal_path), 1):
        # Increment the case number for "正常"
        normal_case_num = disease_to_case_number.get("正常", -1) + 1
        disease_to_case_number["正常"] = normal_case_num
        new_case_number = f"症例{str(normal_case_num).zfill(3)}"

        new_name = f"OSK_正常_{new_case_number}_画像0{os.path.splitext(file_name)[1]}"
        src_file_path = os.path.join(normal_path, file_name)
        dst_file_path = os.path.join(dst_path, new_name)
        shutil.copy2(src_file_path, dst_file_path)

orig_path = r"F:\SlitPhoto\Extracted_tumor_images_sorted"
normal_path = r"F:\SlitPhoto\Extracted_tumor_normal"
dst_path = r"F:\SlitPhoto\Extracted_tumor_images_送付用"

if os.path.exists(dst_path):
    shutil.rmtree(dst_path)
os.makedirs(dst_path)

rename_and_copy_files(orig_path, normal_path, dst_path)
